In [1]:
# 判断鬼吹灯和盗墓笔记以及老九门的相似度
# https://www.cnblogs.com/iloveai/p/gensim_tutorial.html
'''
Gensim是一款开源的第三方Python工具包，用于从原始的非结构化的文本中，无监督地学习到文本隐层的主题向量表达。
它支持包括TF-IDF，LSA，LDA，和word2vec在内的多种主题模型算法，
支持流式训练，并提供了诸如相似度计算，信息检索等一些常用任务的API接口
'''
# api说明
# 建立语料特征，模型(TFIDF)，相似度
from gensim import corpora, models, similarities
import jieba

C:\Users\Just Do It\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# 分词处理
# 导入文件
with open('./老九门.txt', 'r', encoding = 'utf-8') as f_:
    data_ljm = f_.read()
# 引入鬼吹灯
with open('./鬼吹灯.txt', 'r', encoding = 'utf-8') as f_2:
    data_gcd = f_2.read()
# 引入血尸
with open('./血尸.txt', 'r', encoding = 'utf-8') as f_4:
    data_xs = f_4.read()
    
# 分词
data_gcd_op = jieba.cut(data_gcd)
# 处理格式
data_gcd_str = ''
for item in data_gcd_op:
    data_gcd_str += item + ' '

# 分词
data_xs_op = jieba.cut(data_xs)
# 处理格式
data_xs_str = ''
for item in data_xs_op:
    data_xs_str += item + ' '    
    
# 分词并处理格式
data_ljm_op = jieba.cut(data_ljm)
# for item in data_ljm:
#     print(item)
# 拼接起来
# "词语1 词语2 词语3 … 词语n"
data_ljm_str = ''
for item in data_ljm_op:
    data_ljm_str += item + ' '
# print(data_ljm_str)

# 放入列表中
documents = [data_ljm_str, data_gcd_str, data_xs_str]
# print(type(documents))
# 放到二维数组中去
texts = [[word for word in document.split()] for document in documents]
# documents = ['你,好啊', '很好,的']
# texts = [[i for i in item.split()]for item in documents]
# print(texts)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JUSTDO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.868 seconds.
Prefix dict has been built succesfully.


In [3]:
# https://www.liaoxuefeng.com/wiki/001374738125095c955c1e6d8bb493182103fac9270762a000/001411031239400f7181f65f33a4623bc42276a605debf6000
# 使用dict时，如果引用的Key不存在，就会抛出KeyError。如果希望key不存在时，返回一个默认值，就可以用defaultdict：
from collections import defaultdict
# 词频 
frequency = defaultdict(int)
for text in texts:
    for token in text:
        # 计算词出现的频率 {'token(词语)': 'value'}
        frequency[token] += 1
# 将词频低的词语过滤掉
texts = [[token for token in text if frequency[token] > 25] for text in texts]
# print(texts)

In [4]:
# 建立语料特征词典
dictionary = corpora.Dictionary(texts)
# dictionary.save('./12345.txt') 

In [5]:
with open('./盗墓笔记.TXT', 'r', encoding = 'utf-8') as f_1:
    data_dmbj = f_1.read()
data_dmbj_op = jieba.cut(data_dmbj)
data_dmbj_str = ''
for item in data_dmbj_op:
    data_dmbj_str += item + ' '

In [6]:
# 将要对比的文档通过doc2bow转为稀疏向量
data_dmbj_str_vec = dictionary.doc2bow(data_dmbj_str.split())

texts_vec = [dictionary.doc2bow(text) for text in texts]
# 保存稀疏向量语料库
# corpora.MmCorpus.serialize('./6562.txt',texts_vec) 

In [7]:
# 通过tfidfmodel进行处理得到tfidf
tfidf = models.TfidfModel(texts_vec)
# 通过token2id得到特征数
fetureNum = len(dictionary.token2id.keys())
# 稀疏矩阵相似度
# params -> 稀疏向量后的tfidf，特征数
index = similarities.SparseMatrixSimilarity(tfidf[texts_vec], num_features = fetureNum)
# 建立索引，计算相似度
sims = index[tfidf[data_dmbj_str_vec]]
# 血尸的相似度会被稀疏掉
print(sims)

[ 0.23141621  0.55574089  0.03429166]
